<a href="https://colab.research.google.com/github/yujinoh0103/stockChat/blob/main/1016_4%EB%B2%88%EC%A7%B8_%EB%AF%B8%ED%8C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링

[네이버 클러스터링 알고리즘](https://media.naver.com/algorithm)
증권기사크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
from urllib.parse import urlparse, parse_qs

def convert_finance_url_to_news_url(finance_url):
    """
    네이버 금융 뉴스 URL을 일반 뉴스 URL로 변환하는 함수

    Args:
        finance_url (str): 변환할 네이버 금융 뉴스 URL

    Returns:
        str: 변환된 일반 뉴스 URL
    """
    # URL 파싱
    parsed_url = urlparse(finance_url)

    # 쿼리 파라미터 추출
    query_params = parse_qs(parsed_url.query)

    # 필요한 파라미터 추출
    article_id = query_params.get('article_id', [''])[0]
    office_id = query_params.get('office_id', [''])[0]

    # 변환된 URL 생성
    if article_id and office_id:
        new_url = f"https://n.news.naver.com/mnews/article/{office_id}/{article_id}"
        return new_url
    else:
        return "Invalid URL: 필요한 파라미터가 없습니다."

# 사용 예시
finance_url = "https://finance.naver.com/news/news_read.naver?article_id=0005044826&office_id=015&mode=mainnews&type=&date=2024-10-16&page=1"
news_url = convert_finance_url_to_news_url(finance_url)
print(news_url)


https://n.news.naver.com/mnews/article/015/0005044826


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

urls = []

def get_news_urls(start_days_ago=0, end_days_ago=0, max_pages=None):
    """
    날짜 범위와 페이지 수를 기준으로 뉴스 URL 정보를 가져오는 함수

    Args:
        start_days_ago (int): 며칠 전부터 뉴스를 가져올지 결정. (기본값: 0)
        end_days_ago (int): 며칠 전까지 뉴스를 가져올지 결정. (기본값: 0, 즉 하루만 가져옴)
        max_pages (int): 가져올 페이지 수. 기본값은 None(마지막 페이지까지).

    Returns:
        list: 추출한 뉴스 URL 리스트
    """
    global urls

    for day_offset in range(start_days_ago, end_days_ago - 1, -1):
        # 날짜 설정 (오늘 날짜에서 day_offset만큼 이전으로 설정)
        target_date = datetime.now() - timedelta(days=day_offset)
        formatted_date = target_date.strftime('%Y-%m-%d')

        base_url = "https://finance.naver.com/news/mainnews.naver"
        page = 1
        last_page = None  # 마지막 페이지 변수 설정

        while True:
            try:
                params = {'date': formatted_date, 'page': page}
                response = requests.get(base_url, params=params)
                response.raise_for_status()
            except Exception as e:
                break

            # BeautifulSoup을 사용하여 HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # <dd class="articleSubject">에서 뉴스 URL 추출
            articles = soup.find_all('dd', class_='articleSubject')

            # 마지막 페이지를 확인하기 위해 <td class="pgRR">에서 마지막 페이지 링크 추출
            if last_page is None:
                pgRR_tag = soup.find('td', class_='pgRR')
                if pgRR_tag:
                    last_page_link = pgRR_tag.find('a')['href']
                    last_page = int(last_page_link.split('page=')[-1])

            # 각 기사에서 URL 추출
            for article in articles:
                link_tag = article.find('a')
                if link_tag:
                    link = link_tag['href']
                    full_link = "https://n.news.naver.com" + link
                    urls.append(full_link)

            # 마지막 페이지에 도달하면 종료
            if last_page and page >= last_page:
                break

            # 지정된 max_pages에 도달하면 종료
            if max_pages and page >= max_pages:
                break

            page += 1

    return urls

def prompt_user_for_input():
    """
    사용자로부터 날짜 및 페이지 입력을 받고 뉴스 URL을 가져와 출력하는 함수
    """
    # 사용자 입력 받기
    start_days_ago = int(input("시작 날짜: 며칠 전 뉴스부터 가져올까요? (기본값: 0): ") or 0)
    end_days_ago = int(input("끝 날짜: 며칠 전 뉴스까지 가져올까요? (기본값: 0): ") or 0)
    max_pages = input("몇 페이지까지 가져올까요? (기본값: 마지막 페이지): ")
    max_pages = int(max_pages) if max_pages else None

    # 뉴스 URL 가져오기
    news_urls = get_news_urls(start_days_ago=start_days_ago, end_days_ago=end_days_ago, max_pages=max_pages)

    return news_urls


In [ ]:
# 프로그램 실행
prompt_user_for_input()
print('완료')

시작 날짜: 며칠 전 뉴스부터 가져올까요? (기본값: 0): 450
끝 날짜: 며칠 전 뉴스까지 가져올까요? (기본값: 0): 400
몇 페이지까지 가져올까요? (기본값: 마지막 페이지): 


KeyboardInterrupt: 

In [ ]:
final_urls=[]

for finance_url in urls:
    news_url = convert_finance_url_to_news_url(finance_url)
    final_urls.append(news_url)

print(len(final_urls))

## 뉴스를 읽는 함수


In [ ]:
def get_article_content(url):
    """
    주어진 URL에서 기사 제목과 본문 내용을 가져오는 함수

    Args:
        url (str): 가져올 페이지의 URL

    Returns:
        dict: 기사 제목과 본문 내용을 포함하는 딕셔너리
    """
    try:
        # 링크에서 HTML 가져오기
        response = requests.get(url)
        response.raise_for_status()  # 요청 중 에러 발생 시 예외 처리

        # BeautifulSoup을 이용해 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 제목 가져오기
        title_tag = soup.find('h2', id='title_area')
        title = title_tag.get_text(strip=True) if title_tag else "제목을 찾을 수 없습니다."

        # <article> 태그와 id="dic_area"에 해당하는 내용 찾기
        article_body = soup.find('article', id='dic_area')

        # 본문 내용 추출
        if article_body:
            content = article_body.get_text(strip=True)
        else:
            content = "본문을 찾을 수 없습니다."

        # 제목과 본문을 딕셔너리로 반환
        return {
            "title": title,
            "content": content
        }

    except requests.exceptions.RequestException as e:
        return {"error": f"에러 발생: {e}"}


전처리

In [ ]:
def get_article_content(url):
    """
    주어진 URL에서 기사 제목과 본문 내용을 가져오는 함수.
    strong 태그와 이미지 설명(span, em 태그 등)을 제거하고 특수기호를 제거한 텍스트를 반환.

    Args:
        url (str): 가져올 페이지의 URL

    Returns:
        dict: 기사 제목과 본문 내용을 포함하는 딕셔너리
    """
    try:
        # 링크에서 HTML 가져오기
        response = requests.get(url)
        response.raise_for_status()  # 요청 중 에러 발생 시 예외 처리

        # BeautifulSoup을 이용해 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 제목 가져오기
        title_tag = soup.find('h2', id='title_area')
        title = title_tag.get_text(strip=True) if title_tag else "제목을 찾을 수 없습니다."

        # <article> 태그와 id="dic_area"에 해당하는 내용 찾기
        article_body = soup.find('article', id='dic_area')

        # 본문 내용 추출
        if article_body:
            # 불필요한 태그들 (strong, span.end_photo_org, em 등) 제거
            for tag in article_body.find_all(['strong', 'img']):
                tag.decompose()  # 해당 태그와 그 내부 내용을 제거

            # 클래스 속성이 있는 태그들을 모두 제거
            for tag_with_class in article_body.find_all(True, class_=True):
                tag_with_class.decompose()  # 해당 태그를 제거

            # 텍스트에서 특수기호 제거
            content = article_body.get_text(strip=True)
            content = remove_special_characters(content)  # 특수 기호 제거 함수 사용

            for br_tag in article_body.find_all('br'):
                br_tag.replace_with("  ")
        else:
            content = "본문을 찾을 수 없습니다."

        # 제목과 본문을 딕셔너리로 반환
        return {
            "title": title,
            "content": content
        }

    except requests.exceptions.RequestException as e:
        return {"error": f"에러 발생: {e}"}

def remove_special_characters(text):
    """
    텍스트에서 특수기호나 불필요한 문자를 제거하는 함수

    Args:
        text (str): 원본 텍스트

    Returns:
        str: 특수문자가 제거된 텍스트
    """
    # 불필요한 특수 기호 제거 (네모 기호 등)
    unwanted_chars = ['\u25a0', '\u25a1', '\u00a0']  # 추가로 제거할 기호가 있으면 여기에 추가
    for char in unwanted_chars:
        text = text.replace(char, '')

    return text


In [ ]:
import json

In [ ]:

def save_to_json(data, filename):
    """
    데이터를 JSON 파일로 저장하는 함수

    Args:
        data (dict): 저장할 데이터
        filename (str): 저장할 파일 이름
    """
    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)  # UTF-8로 저장, 가독성을 위해 indent 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


런타임 끝나서 저장해둔 json에서 읽어오기

In [ ]:
import json

# JSON 파일에서 리스트 읽어오기
with open('/content/drive/MyDrive/단기연구/final_urls.json', 'r', encoding='utf-8') as json_file:
    final_urls = json.load(json_file)

In [ ]:
len(final_urls)

8355

In [ ]:
all_articles = []

for url in final_urls:
    content = get_article_content(url)
    all_articles.append(content)

# 하나의 JSON 파일에 모든 기사 저장
save_to_json(all_articles, 'all_articles.json')
print("Saved all articles to all_articles.json")

KeyboardInterrupt: 

In [ ]:
save_to_json(all_articles, 'all_articles.json')
print("Saved all articles to all_articles.json")

Saved all articles to all_articles.json


In [ ]:
len(all_articles)

6105

In [ ]:
import json
import re

# JSON 파일 경로
file_path = 'all_articles.json'

# JSON 파일을 읽고 데이터를 로드합니다.
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 괄호와 그 안의 내용 제거 함수
def remove_bracket_contents(text):
    # 정규표현식을 사용하여 괄호와 그 안의 내용 제거
    return re.sub(r"[【\[].*?[】\]]", "", text).strip()

# JSON 데이터 안의 모든 문자열에서 괄호와 그 안의 내용 제거
def clean_json(data):
    if isinstance(data, dict):
        for key, value in data.items():
            data[key] = clean_json(value)
    elif isinstance(data, list):
        for i in range(len(data)):
            data[i] = clean_json(data[i])
    elif isinstance(data, str):
        return remove_bracket_contents(data)
    return data

# 데이터 정리
cleaned_data = clean_json(data)

# 정리된 데이터를 새로운 파일에 저장
with open('cleaned_data.json', 'w', encoding='utf-8') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("JSON 파일에서 괄호와 내용이 제거되었습니다.")


JSON 파일에서 괄호와 내용이 제거되었습니다.
